In [39]:
import numpy as np

In [40]:
import wandb

# Initialize a wandb API object
api = wandb.Api(timeout=60)

# Replace 'your_project_path' with your actual project path, e.g., 'evangeorgerex/f


In [41]:
run = api.run("evangeorgerex/fwal/runs/xgzcuu37")
# xgzcuu37 xgboost
# 2j5c9lxu lasso
# 9on5gumd rf

In [49]:
def load_feature_importance(run):
    # Load the feature importance
    history = run.history()
    
    max_index = max(int(col.split('_')[-1]) for col in history.columns if 'feature_importances.feature' in col)
    feature_importances = np.full(max_index + 1, np.nan)

    # Iterate over each column and place the non-NaN value in the corresponding position in the array
    for col in history.columns:
        if 'feature_importances.feature' in col:
            # Extract the feature index from the column name
            feature_index = int(col.split('_')[-1])
            # Find the non-NaN value in this column
            non_nan_value = history[col].dropna().values[0]  # assuming there is exactly one non-NaN value per feature
            feature_importances[feature_index] = non_nan_value
    return feature_importances

def load_feature_importance_lasso(run):

    history = run.history()

    # Determine the maximum indices for classes and features
    max_class_index = max(int(col.split('_')[1]) for col in history.columns if 'class_' in col)
    max_feature_index = max(int(col.split('_')[-1]) for col in history.columns if 'feature' in col)

    # Initialize a 2D array of NaNs
    coef = np.full((max_class_index + 1, max_feature_index + 1), np.nan)

    # Iterate over each column and assign values to the coef array
    for col in history.columns:
        if 'class_' in col and 'feature' in col:
            parts = col.split('_')
            class_index = int(parts[1])
            feature_index = int(parts[-1])
            # Find the non-NaN value in this column
            non_nan_value = history[col].dropna().values[0]  # assuming there is exactly one non-NaN value per feature and class
            coef[class_index, feature_index] = non_nan_value

    # Now 'coef' is your reconstructed 2D array similar to 'model.coef_' in sklearn Lasso for multi-class problems
    feature_importance = np.mean(np.abs(coef), axis=0)
    return feature_importance

In [43]:
load_feature_importance(run)

array([0.006493, 0.      , 0.      , ..., 0.      , 0.      , 0.      ])

In [44]:
rf_run = api.run("evangeorgerex/fwal/runs/9on5gumd")
load_feature_importance(rf_run)


array([0.00398822, 0.0020938 , 0.00098478, ..., 0.00302319, 0.00384115,
       0.00315465])

In [45]:
lasso_run = api.run("evangeorgerex/fwal/runs/2j5c9lxu")


In [50]:
load_feature_importance_lasso(lasso_run)

array([0.02696365, 0.0113033 , 0.00118566, ..., 0.01905022, 0.0274675 ,
       0.02776831])